## Загрузка модели

In [1]:
!pip install transformers sentencepiece torch --quiet

In [39]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

model_name = "sarahai/ruT5-base-summarizer"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name).to("cuda")
device = torch.device("cuda")

In [67]:
def inference(input_text):
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)
    outputs = model.generate(input_ids, max_length=300, min_length=50, length_penalty=1.0, num_beams=6, early_stopping=True)
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return summary

## Тестим на текстах

In [80]:
import json

with open("example_texts.json", "r") as f:
    texts = json.loads(f.read())
texts[0][:300]

'Увидеть мысль\n\nЯпонским ученым удалось распознать изображение , увиденное человеком , сканируя его мозг .\n\nКак пишет Yomiuri , группе сотрудников отделения нейроинформатики Международного исследовательского института передовых средств коммуникации ( Киото , Япония ) удалось восстановить изображение '

In [83]:
print(f"Input text length: {len(texts[0])}")
print(f"\nInput text: " + texts[0][:300] + "...")
print(f"\nSummary res: {inference(texts[0])}")

Input text length: 1213

Input text: Увидеть мысль

Японским ученым удалось распознать изображение , увиденное человеком , сканируя его мозг .

Как пишет Yomiuri , группе сотрудников отделения нейроинформатики Международного исследовательского института передовых средств коммуникации ( Киото , Япония ) удалось восстановить изображение ...

Summary res: Японские ученые смогли распознать изображение, увиденное человеком, сканируя его мозг. Это удалось сделать благодаря сканированию электрических сигналов мозга. Исследователи надеются, что их работа поможет им в реализации идеи прямого человеко-машинного интерфейса.


In [85]:
print(f"Input text length: {len(texts[3])}")
print(f"\nInput text: " + texts[3][:300] + "...")
print(f"\nSummary res: {inference(texts[3])}")

Input text length: 16098

Input text: « Лужков . Итоги »
Конец эры хозяйственника или пиар для Немцова ?

Опубликованный доклад Бориса Немцова « Лужков . Итоги » вызвал немалый шум среди общественности .
Мэр города Москвы сделал заявление о том , что одной из важнейших задач в деле модернизации России должно стать сокращение чиновничьег...

Summary res: Уход Лужкова с поста мэра Москвы, как и в случае с другими чиновниками, может привести к тому, что избиратели, и чиновники, и чиновники, и граждане будут вынуждены платить за ЖКХ, а не за ЖКХ, как это было при Ельцине.


Заметим плохую обработку текста большого размера. Попробуем другой подход путем разбиения на параграфы и конкатенацию реферата.

In [88]:
def split_text_for_summarization(text, max_chunk_size=2500):
    """
    Умное разбиение текста на части для суммаризации
    """
    # Разбиваем по абзацам (двойной перенос строки)
    paragraphs = text.split('\n\n')

    chunks = []
    current_chunk = ""

    for paragraph in paragraphs:
        # Если добавление нового абзаца не превысит лимит
        if len(current_chunk) + len(paragraph) + 2 <= max_chunk_size:
            current_chunk += paragraph + "\n\n"
        else:
            # Если текущий чанк не пустой, сохраняем его
            if current_chunk:
                chunks.append(current_chunk.strip())
            # Начинаем новый чанк с текущего абзаца
            current_chunk = paragraph + "\n\n"

    # Добавляем последний чанк
    if current_chunk:
        chunks.append(current_chunk.strip())

    return chunks

In [ ]:
def hierarchical_summarization(text):
    """
    Иерархическая суммаризация: сначала по частям, потом общий итог
    """
    # Первый уровень: разбиваем на крупные части
    large_chunks = split_text_for_summarization(text, max_chunk_size=4000)

    first_level_summaries = []

    for chunk in large_chunks:
        # Второй уровень: разбиваем крупные части на меньшие
        small_chunks = split_text_for_summarization(chunk, max_chunk_size=2000)

        chunk_summaries = []
        for small_chunk in small_chunks:
            small_summary = inference(small_chunk)
            chunk_summaries.append(small_summary)

        # Объединяем суммаризации для крупной части
        combined_chunk_summary = " ".join(chunk_summaries)
        first_level_summaries.append(combined_chunk_summary)

    # Финальная суммаризация объединенных результатов
    final_combined = " ".join(first_level_summaries)

    # Если финальный текст все еще слишком длинный, делаем финальную суммаризацию
    if len(final_combined) > 4000:
        final_summary = inference(final_combined)
    else:
        final_summary = final_combined
    return final_summary

In [ ]:
def batch_hierarchical_summarization(texts):
    summaries = []

    for text in texts:
        summary = hierarchical_summarization(text)
        summaries.append(summary)
    return summaries

In [91]:
print(f"Input text length: {len(texts[1])}")
print(f"\nInput text: " + texts[1][:300] + "...")
summary = hierarchical_summarization(texts[1])
print(f"Summary length: {len(summary)}")
print(f"\nSummary res: {summary}")

Input text length: 928

Input text: Карбофос

Карбофос ( O , O - Диметил - S - ( 1,2 - дикарбэтоксиэтил ) дитиофосфат , малатион ) — фосфорорганическое соединение , инсектицид широкого спектра действия , акарицид .

Применение

Применялся против комаров , мух , клещей , клопов , паразитов , повреждающих фруктовые деревья , овощные и д...
Summary length: 190

Summary res: Антихолинэстеразное средство необратимого действия, инактивирующее ацетилхолинэстеразное средство необратимого действия. Механизм действия Антихолинэстеразное средство необратимого действия.


In [90]:
print(f"Input text length: {len(texts[3])}")
print(f"\nInput text: " + texts[3][:300] + "...")
summary = hierarchical_summarization(texts[3])
print(f"Summary length: {len(summary)}")
print(f"\nSummary res: {summary}")

Input text length: 16098

Input text: « Лужков . Итоги »
Конец эры хозяйственника или пиар для Немцова ?

Опубликованный доклад Бориса Немцова « Лужков . Итоги » вызвал немалый шум среди общественности .
Мэр города Москвы сделал заявление о том , что одной из важнейших задач в деле модернизации России должно стать сокращение чиновничьег...
Summary length: 2816

Summary res: Итоги работы мэра Москвы Юрия Лужкова вызвали бурную бизнес-деятельность его жены и роль Лужкова в этой деятельности. С такими результатами мэр Москвы должен быть немедленно отправлен в отставку, а его бизнес-деятельность и роль в этой деятельности должны быть тщательно расследованы и преданы гласности. Немыслимый доклад Бориса Немцова о деятельности Юрия Лужкова свидетельствует о том, что «крепкий хозяйственник» успел побыть и оппонентом, и партнёром всех российских бизнес-структур первого уровня, и о том, как его признали достойным партнёром все участники российского списка Forbes. Столичные москвичи, судя по всем

## Считаем метрики

In [48]:
!pip install evaluate rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=441c7de90c7abc1cfd121efdfbd52679e5ab0add1006e05215c211a72802ffc6
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score


In [92]:
from datasets import load_dataset
import evaluate
from tqdm import tqdm

rouge = evaluate.load('rouge')
test_split = load_dataset("RussianNLP/Mixed-Summarization-Dataset", split='test')

def clean_text(example):
    start_ind = example["text"].find('"')
    example["text"] = (example["text"][start_ind+1:len(example['text'])-1]).replace('\n','')
    return example

references_test = []
summaries_test = []

test_split = test_split.map(clean_text)
for example in tqdm(test_split):
    reference = example['text']
    summary = hierarchical_summarization(reference)[:300]
    references_test.append(reference)
    summaries_test.append(summary)

result = rouge.compute(
    predictions = summaries_test,
    references = references_test,
    tokenizer=lambda x: x.split()
)
print(result)

Map:   0%|          | 0/258 [00:00<?, ? examples/s]

100%|██████████| 258/258 [06:24<00:00,  1.49s/it]


{'rouge1': np.float64(0.23985401908299248), 'rouge2': np.float64(0.16909789209697168), 'rougeL': np.float64(0.2089122375916892), 'rougeLsum': np.float64(0.2092078851779524)}
